In [26]:
import cv2
import pytesseract
import numpy as np

# Set the Tesseract path (update this with your Tesseract installation path)
pytesseract.pytesseract.tesseract_cmd = r'C:\Programs\Tesseract\tesseract.exe'


In [129]:
def find_largest_contour(contours, height, width):
    return max(
        contours,
        key=cv2.contourArea,
        default=[
            np.array([[0, 0]]),
            np.array([[0, height]]),
            np.array([[width, height]]),
            np.array([[width, 0]]),
        ],
    )


def get_corners_from_contour(contour):
    peri = cv2.arcLength(contour, True)
    approx = cv2.approxPolyDP(contour, 0.02 * peri, True)
    return np.squeeze(approx)


def order_points(corners):
    rect = np.zeros((4, 2), dtype=np.float32)
    s = corners.sum(axis=1)
    rect[0] = corners[np.argmin(s)]
    rect[2] = corners[np.argmax(s)]
    diff = np.diff(corners, axis=1)
    rect[1] = corners[np.argmin(diff)]
    rect[3] = corners[np.argmax(diff)]
    return rect


def get_warped_image(image, corners):
    width, height = 450, 450
    rect = order_points(corners)
    dst = np.array(
        [[0, 0], [width - 1, 0], [width - 1, height - 1], [0, height - 1]],
        dtype=np.float32,
    )
    M = cv2.getPerspectiveTransform(rect, dst)
    warped = cv2.warpPerspective(image, M, (width, height))
    return warped


def divide_into_cells(image):
    rows, cols = 9, 9
    cell_height, cell_width = image.shape[0] // rows, image.shape[1] // cols
    cells = []

    for r in range(rows):
        for c in range(cols):
            y1, y2 = r * cell_height, (r + 1) * cell_height
            x1, x2 = c * cell_width, (c + 1) * cell_width
            cells.append(image[y1:y2, x1:x2])

    return cells


def remove_cell_border(cell_image):
    # Remove the outer 10 pixels from each side of the cell image
    borderSize = 5
    cell_no_border = cell_image[borderSize:-borderSize, borderSize:-borderSize]

    return cell_no_border


In [166]:
preprocessed_image = cv2.imread("sudoku_picture.jpg", cv2.IMREAD_GRAYSCALE)

 # Apply Gaussian blur to reduce noise
blurred = cv2.GaussianBlur(preprocessed_image, (5, 5), 0)

# Apply adaptive thresholding to create a binary image
_, thresh = cv2.threshold(blurred, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

# Perform morphological operations to remove noise and smooth the image
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
cleaned = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel)

# Find contours in the image
contours, _ = cv2.findContours(cleaned.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
# contours, _ = cv2.findContours(preprocessed_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Find the largest contour (the outer boundary of the Sudoku grid)
height, width = preprocessed_image.shape[:2]
sudoku_grid_contour = find_largest_contour(contours, height, width)

# Get the corners of the Sudoku grid
sudoku_grid_corners = get_corners_from_contour(sudoku_grid_contour)

# Warp the image to get a top-down view of the Sudoku grid
# warped_image = get_warped_image(cleaned, sudoku_grid_corners)
warped_image = get_warped_image(preprocessed_image, sudoku_grid_corners)

# Divide the warped image into individual cells
sudoku_cells = divide_into_cells(warped_image)

In [167]:
preprocessed_image = cv2.imread("sudoku_picture.jpg", cv2.IMREAD_GRAYSCALE)
# height, width = preprocessed_image.shape[:2]
# height, width

output_file = "cleaned.png"
cv2.imwrite(output_file, cleaned)

True

In [154]:
cells_without_border = [remove_cell_border(cell) for cell in sudoku_cells]

In [162]:
image = preprocessed_image
corners = sudoku_grid_corners

width, height = 450, 450
rect = order_points(corners)
dst = np.array(
    [[0, 0], [width - 1, 0], [width - 1, height - 1], [0, height - 1]],
    dtype=np.float32,
)
M = cv2.getPerspectiveTransform(rect, dst)
warped = cv2.warpPerspective(image, M, (width, height))


warped_image = get_warped_image(preprocessed_image, sudoku_grid_corners)

In [165]:
def cut_out_region(image, corners):
    # Get the bounding rectangle of the region of interest (ROI)
    x, y, width, height = cv2.boundingRect(corners)

    # Extract the portion of the image within the ROI
    roi = image[y:y+height, x:x+width]

    return roi

cutout_image = cut_out_region(image, corners)


output_file = "selection.png"
cv2.imwrite(output_file, cutout_image)

True

In [160]:
output_file = "preprocessed_image.png"
cv2.imwrite(output_file, preprocessed_image)

output_file = "warped_image.png"
cv2.imwrite(output_file, warped_image)

output_file = "cells_7.png"
cv2.imwrite(output_file, sudoku_cells[7])

output_file = "cells_8.png"
cv2.imwrite(output_file, sudoku_cells[8])

output_file = "cells_8_no_border.png"
cv2.imwrite(output_file, cells_without_border[8])

output_file = "cells_0_no_border.png"
cv2.imwrite(output_file, cells_without_border[0])

True

In [97]:
sudoku_array = [[0 for _ in range(9)] for _ in range(9)]

# Perform OCR on each cell to recognize the digits
for row in range(9):
    for col in range(9):
        # Get the cell image
        cell_image = cells_without_border[row * 9 + col]

        # Apply OCR using Tesseract (you need to have pytesseract installed)
        digit = pytesseract.image_to_string(
            cell_image, config="--psm 10 --oem 3 -c tessedit_char_whitelist=123456789"
        )
        # Remove unwanted characters and strip whitespace, including newline character
        cleaned_digit = digit.strip()

        # Store the recognized digit in the Sudoku array
        sudoku_array[row][col] = int(cleaned_digit) if cleaned_digit.isdigit() else 0

In [98]:
sudoku_array

[[1, 0, 0, 0, 0, 0, 0, 0, 6],
 [0, 0, 6, 0, 2, 0, 7, 0, 0],
 [7, 8, 9, 4, 5, 0, 1, 0, 3],
 [0, 0, 0, 8, 0, 7, 0, 0, 4],
 [0, 0, 0, 0, 3, 0, 0, 0, 0],
 [0, 9, 0, 0, 0, 4, 2, 0, 1],
 [3, 1, 2, 9, 7, 0, 0, 4, 0],
 [0, 4, 0, 0, 1, 2, 0, 7, 8],
 [9, 0, 8, 0, 0, 0, 0, 0, 0]]

In [159]:
number = 3
cell_image = cells_without_border[number]
digit = pytesseract.image_to_string(
            cell_image, config="--psm 10 --oem 3 -c tessedit_char_whitelist=123456789"
        )
print(digit)
output_file = "cells_0_no_border.png"
cv2.imwrite(output_file, cells_without_border[number])

7



True